In [1]:
# Spot6 Stereo Pipeline using Ames Stereo Pipeline
# ------------------------------------------------
# Requirements:
# - Install Ames Stereo Pipeline:"https://stereopipeline.readthedocs.io/en/latest/installation.html#conda-intro"
# - Make sure TIF and XML files are extracted and available
# - DEM file available for optional mapprojection

import os
from IPython.display import Markdown

In [15]:

# Set paths for input images and RPCs
LEFT_IMG = "data/tif/front/IMG_SPOT6_P_201906210832287_SEN_7416126101_R1C1.TIF"
RIGHT_IMG = "data/tif/back/IMG_SPOT6_P_201906210833144_SEN_7416126101_R1C1.TIF"

LEFT_RPC = "data/tif/front/RPC_SPOT6_P_201906210832287_SEN_7416126101.XML"
RIGHT_RPC = "data/tif/back/RPC_SPOT6_P_201906210833144_SEN_7416126101.XML"

DEM_REF = "data/tif/dem.tif"
STEREO_PREFIX = "results/out"


front_map_proj="results/front_map_proj.tif"
back_map_proj="results/back_map_proj.tif"

Markdown(f"""
### Input Configuration:
- Left Image: `{LEFT_IMG}`
- Right Image: `{RIGHT_IMG}`
- Left RPC: `{LEFT_RPC}`
- Right RPC: `{RIGHT_RPC}`
- DEM Reference: `{DEM_REF}`
- Stereo Output Prefix: `{STEREO_PREFIX}`
- Front Map Projection Output: `{front_map_proj}`
- Back Map Projection Output: `{back_map_proj}`
""")


### Input Configuration:
- Left Image: `data/tif/front/IMG_SPOT6_P_201906210832287_SEN_7416126101_R1C1.TIF`
- Right Image: `data/tif/back/IMG_SPOT6_P_201906210833144_SEN_7416126101_R1C1.TIF`
- Left RPC: `data/tif/front/RPC_SPOT6_P_201906210832287_SEN_7416126101.XML`
- Right RPC: `data/tif/back/RPC_SPOT6_P_201906210833144_SEN_7416126101.XML`
- DEM Reference: `data/tif/dem.tif`
- Stereo Output Prefix: `results/out`
- Front Map Projection Output: `results/front_map_proj.tif`
- Back Map Projection Output: `results/back_map_proj.tif`


# Step 1 – Optional: Mapproject Both Images

In [ ]:
# !mapproject -t rpc {DEM_REF} {LEFT_IMG}  {LEFT_RPC}  {front_map_proj}
# !mapproject -t rpc {DEM_REF} {RIGHT_IMG} {RIGHT_RPC} {back_map_proj}

In [8]:
# Force same GSD (e.g., 1.5 meters per pixel for SPOT6)
mpp = 1.5

!mapproject -t rpc --mpp {mpp} {DEM_REF} {LEFT_IMG}  {LEFT_RPC}  {front_map_proj}
!mapproject -t rpc --mpp {mpp} {DEM_REF} {RIGHT_IMG} {RIGHT_RPC} {back_map_proj}

mapproject_single --query-projection data/tif/dem.tif data/tif/front/IMG_SPOT6_P_201906210832287_SEN_7416126101_R1C1.TIF data/tif/front/RPC_SPOT6_P_201906210832287_SEN_7416126101.XML results/front_map_proj.tif -t rpc --mpp 1.5
	--> Setting number of processing threads to: 4
Using session: rpc
Loading camera model: data/tif/front/IMG_SPOT6_P_201906210832287_SEN_7416126101_R1C1.TIF data/tif/front/RPC_SPOT6_P_201906210832287_SEN_7416126101.XML
Output pixel size: 1.5
Projected space bounding box: Min: (333727.5, 3255255) width: 6070.5 height: 6243
Output image size:
(width: 4048 height: 4163)
Query finished, exiting mapproject tool.

Output image size is 4048 by 4163 pixels.
Splitting into 1 by 1 tiles.
parallel --will-cite -u --env PATH --env PYTHONPATH --env ISISROOT --env ASP_LIBRARY_PATH --env ISISDATA -a results/front_map_proj_tif_tiles/argumentList.txt -P 1 --colsep \t --sshdelay 0.2 /home/akshay_wsl/miniconda3/envs/asp/bin/python /home/akshay_wsl/miniconda3/envs/asp/bin/mapproject -

| Output File          | Description                                                   |
| -------------------- | ------------------------------------------------------------- |
| `front_map_proj.tif` | Left image resampled and orthorectified using the DEM and RPC |
| `back_map_proj.tif`  | Right image resampled similarly                               |

These are georeferenced images aligned to the same DEM grid and ready for stereo.

# Step 2 – Stereo Matching using rpcmaprpc Model

In [12]:
!parallel_stereo -t rpcmaprpc \
    {front_map_proj} {back_map_proj} \
    {LEFT_RPC} {RIGHT_RPC} \
    {STEREO_PREFIX} {DEM_REF}

Using tiles (before collar addition) of 2048 x 2048 pixels.
Using a collar (padding) for each tile of 0 pixels.

[ 2025-Jun-12 17:41:33 ]: Stage 0 --> PREPROCESSING
	--> Setting number of processing threads to: 22
Stereo file ./stereo.default could not be found. Will use default settings and command line options only.
Writing log: results/out-log-stereo_pprc-06-12-1741-34903.txt
Using session: rpcmaprpc
Images before mapprojection: data/tif/front/IMG_SPOT6_P_201906210832287_SEN_7416126101_R1C1.TIF data/tif/back/IMG_SPOT6_P_201906210833144_SEN_7416126101_R1C1.TIF
Mapprojection cameras: data/tif/front/RPC_SPOT6_P_201906210832287_SEN_7416126101.XML data/tif/back/RPC_SPOT6_P_201906210833144_SEN_7416126101.XML
Mapprojected images bundle adjustment prefixes:  
Mapprojection cam types: rpc rpc
Loading camera model: results/front_map_proj.tif data/tif/front/RPC_SPOT6_P_201906210832287_SEN_7416126101.XML
Loading camera model: results/back_map_proj.tif data/tif/back/RPC_SPOT6_P_201906210833144_S

| Output File               | Description                                                          |
| ------------------------- | -------------------------------------------------------------------- |
| `out-L.tif`               | Left mapprojected image cropped and aligned                          |
| `out-R.tif`               | Right mapprojected image cropped and aligned                         |
| `out-D.tif`               | Disparity map (horizontal + vertical offsets between image pairs)    |
| `out-F.tif`               | Correlation score for each pixel in the disparity map                |
| `out-PC.tif`              | **Point cloud** in world coordinates derived from disparity and RPCs |
| `out-IntersectionErr.tif` | Pixel-wise triangulation error estimate                              |
| `out-log-stereo*.txt`     | Stereo log file with all parameters and runtime info                 |

This step performs image matching and 3D triangulation. It generates a raw point cloud and associated data.

/file

# Step 3 – Convert Point Cloud to DEM

In [13]:
!point2dem {STEREO_PREFIX}-PC.tif \
    --orthoimage {STEREO_PREFIX}-L.tif \
    -o {STEREO_PREFIX}/DEM_PC.tif

	--> Setting number of processing threads to: 4
Writing log: results/out/DEM_PC.tif-log-point2dem-06-12-1744-48043.txt
Bounding box and triangulation error range estimation: [******************] 99%
Point cloud extent estimation: [******************************************] 95%
Collected a sample of 14992581 positive triangulation errors.
Error percentiles: Q1 (25%): 0.336456, Q2 (50%): 0.557556, Q3 (75%): 0.817108.
Computing triangulation error cutoff based on --remove-outliers-params.
Triangulation error cutoff is 2.45132 meters.
	Starting DEM rasterization
	--> DEM spacing: 1.42984 pt/px
	             or: 0.699381 px/pt
Creating output file that is Vector2(4131,4245) px.
Writing: results/out/DEM_PC.tif-DEM.tif
DEM: [********************************************************************] 100%
Percentage of valid pixels: 93.9546%
Writing: results/out/DEM_PC.tif-DRG.tif
DRG: [********************************************************************] 100%


| Output File            | Description                                                            |
| ---------------------- | ---------------------------------------------------------------------- |
| `DEM_PC.tif`           | **Digital Elevation Model (DEM)** rasterized from the 3D point cloud   |
| `DEM_PC-hillshade.tif` | Optional visualization-friendly shaded relief from DEM (see next step) |
| `log-point2dem-*.txt`  | Log file of DEM generation                                             |

This step interpolates the 3D point cloud to a regular grid DEM (elevation raster).

*-DRG.tif is a visual rendering of the orthoimage draped over the DEM surface.

It's essentially the input orthoimage (like out-L.tif) reprojected and warped to align with the DEM.

The purpose is to visually verify that the orthorectification and elevation modeling worked correctly.

#  Step 4 – Hillshade for Visualization (Optional)

In [14]:
!hillshade {STEREO_PREFIX}/DEM_PC.tif-DEM.tif -o {STEREO_PREFIX}/DEM_PC_hillshade.tif

	--> Setting number of processing threads to: 4
Loading: results/out/DEM_PC.tif-DEM.tif.
	--> Extracted nodata value from file: -1e+06.
Writing shaded relief image: results/out/DEM_PC_hillshade.tif
Writing:[*****************************************************************] 100%


| File                      | Description                   |
| ------------------------- | ----------------------------- |
| `out-PC.tif`              | Point cloud                   |
| `out-DEM_PC.tif`          | DEM from point cloud          |
| `out-L.tif` / `out-R.tif` | Rectified stereo images       |
| `DEM_PC_hillshade.tif`    | Hillshade from DEM (optional) |

This creates a grayscale shaded relief map from your DEM, helpful for quickly checking terrain shape